In [343]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gender_submission.csv', 'test.csv', 'train.csv']


In [344]:
df = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [345]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

In [346]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [347]:
df.drop(['Name'], axis=1, inplace = True)
df.drop(['PassengerId'], axis=1, inplace= True)
test.drop(['Name'], axis=1, inplace = True)


In [348]:
df.fillna(0, inplace = True)
test.fillna(0, inplace = True)


In [349]:
# Bins for ages
# Labels to categorize the ages.
bins = [0,1, 5, 10, 25, 50, 100]
labels = [1,2,3,4,5,6]

In [350]:
df['Age'] = pd.cut(df['Age'], bins = bins, labels = labels)
test['Age'] = pd.cut(test['Age'], bins = bins, labels = labels)


In [351]:
df = pd.get_dummies(df, columns = ['Sex'],drop_first = True )
test = pd.get_dummies(test, columns = ['Sex'],drop_first = True )

In [352]:
# Creating labels and bins for fare column
bins = [0,10,20,30,50, 100, 200 , 250, 300, 350, 400, 450, 500, 550]
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13]
df['Fare'] = pd.cut(df['Fare'], bins = bins, labels = labels)
test['Fare'] = pd.cut(test['Fare'], bins = bins, labels = labels)
df['Fare'] = df['Fare'].astype('int32')
test['Fare'] =  test['Fare'].astype('int32')

In [353]:
fare_scale = preprocessing.MinMaxScaler()

df_fares = df['Fare'].values
scaled_fares = df_fares.reshape(-1,1)
scaled_fares = fare_scale.fit_transform(scaled_fares)
scaled_fares = scaled_fares.flatten()
df['Fare'] = pd.Series(scaled_fares)

test_fares = test['Fare'].values
scaled_fares = test_fares.reshape(-1,1)
scaled_fares = fare_scale.transform(scaled_fares)
scaled_fares = scaled_fares.flatten()
test['Fare'] = pd.Series(scaled_fares)

In [354]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
0,0,3,4,1,0,A/5 21171,1.0,0,S,1
1,1,1,5,1,0,PC 17599,1.0,C85,C,0
2,1,3,5,0,0,STON/O2. 3101282,1.0,0,S,0
3,1,1,5,1,0,113803,1.0,C123,S,0
4,0,3,5,0,0,373450,1.0,0,S,1


In [355]:
label_enc = preprocessing.LabelEncoder()
df['Cabin'] = df['Cabin'].astype('str')
test['Cabin'] = test['Cabin'].astype('str')

enc_list = []
for i in df['Cabin'].values:
    enc_list.append(i)
    
for i in test['Cabin'].values:
    enc_list.append(i)
    
label_enc.fit(enc_list)
    
df['Cabin'] = label_enc.transform(df['Cabin'])

test['Cabin'] = label_enc.transform(test['Cabin'])

In [356]:
label_enc = preprocessing.LabelEncoder()
df['Embarked'] = df['Embarked'].astype('str')
df['Embarked'] = label_enc.fit_transform(df['Embarked'])

test['Embarked'] = test['Embarked'].astype('str')
test['Embarked'] = label_enc.transform(test['Embarked'])

In [357]:
label_enc = preprocessing.LabelEncoder()
df['Ticket'] = df['Ticket'].astype('str')
test['Ticket'] = test['Ticket'].astype('str')

enc_list = []
for i in df['Ticket'].values:
    enc_list.append(i)
    
for i in test['Ticket'].values:
    enc_list.append(i)
    
label_enc.fit(enc_list)
    
df['Ticket'] = label_enc.transform(df['Ticket'])

test['Ticket'] = label_enc.transform(test['Ticket'])

In [358]:
df = df.astype('float')
df.fillna(0, inplace = True)

test = test.astype('float')
test.fillna(0, inplace = True)

In [359]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.00000,891.000000,891.000000
mean,0.383838,2.308642,3.603816,0.523008,0.381594,462.114478,0.983165,22.61055,2.529742,0.647587
std,0.486592,0.836071,2.000884,1.102743,0.806057,277.025043,0.128725,48.66263,0.800254,0.477990
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.000000,2.000000,3.000000,0.000000,0.000000,211.500000,1.000000,0.00000,2.000000,0.000000
50%,0.000000,3.000000,4.000000,0.000000,0.000000,464.000000,1.000000,0.00000,3.000000,1.000000
75%,1.000000,3.000000,5.000000,1.000000,0.000000,715.000000,1.000000,0.00000,3.000000,1.000000
max,1.000000,3.000000,6.000000,8.000000,6.000000,928.000000,1.000000,186.00000,3.000000,1.000000


In [360]:
X = np.array(df.drop(['Survived'], axis = 1))
X = preprocessing.scale(X)
# If we avoid this preprocessing then it will hurt a lot in SVM
Y = np.array(df['Survived'])

In [429]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,stratify = Y, test_size = 0.1, random_state = 31)

In [434]:
model = svm.SVC(kernel = 'poly',degree=3, random_state = 31, gamma = "auto", C = 1)
model.fit(X_train, Y_train)
predictions = model.predict(X_train)
accuracy = accuracy_score(predictions,Y_train)
print("Training accuracy = %0.2f" %(accuracy * 100))

Training accuracy = 82.90


In [435]:
predictions = model.predict(X_test)
accuracy = accuracy_score(predictions,Y_test)
print("Testing accuracy = %0.2f" %(accuracy * 100))

Testing accuracy = 81.11


In [436]:
clf = DecisionTreeClassifier(max_depth=4, random_state = 31)
clf.fit(X_train, Y_train)
predictions = clf.predict(X_train)
accuracy_clf = accuracy_score(predictions, Y_train)
print("Training accuracy = %0.2f" %(accuracy_clf * 100))

Training accuracy = 83.65


In [437]:
predictions = clf.predict(X_test)
accuracy_clf = accuracy_score(predictions, Y_test)
print("Testing accuracy = %0.2f" %(accuracy_clf * 100))

Testing accuracy = 83.33


In [366]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
0,892.0,3.0,5.0,0.0,0.0,376.0,1.0,0.0,2.0,1.0
1,893.0,3.0,5.0,1.0,0.0,582.0,1.0,0.0,3.0,0.0
2,894.0,2.0,6.0,0.0,0.0,184.0,1.0,0.0,2.0,1.0
3,895.0,3.0,5.0,0.0,0.0,366.0,1.0,0.0,3.0,1.0
4,896.0,3.0,4.0,1.0,1.0,338.0,1.0,0.0,3.0,0.0


In [367]:
Pid = test['PassengerId']
X_final_test = test.drop(['PassengerId'], axis = 1)
X_final_test = preprocessing.scale(X_final_test)


In [368]:
Y_pred = clf.predict(X_final_test)

In [369]:
test['Survived'] = Y_pred

In [370]:
test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male,Survived
0,892.0,3.0,5.0,0.0,0.0,376.0,1.0,0.0,2.0,1.0,0.0
1,893.0,3.0,5.0,1.0,0.0,582.0,1.0,0.0,3.0,0.0,0.0
2,894.0,2.0,6.0,0.0,0.0,184.0,1.0,0.0,2.0,1.0,0.0
3,895.0,3.0,5.0,0.0,0.0,366.0,1.0,0.0,3.0,1.0,0.0
4,896.0,3.0,4.0,1.0,1.0,338.0,1.0,0.0,3.0,0.0,0.0


In [371]:
test = test.astype('int32')

In [372]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": test["Survived"]
    })

In [373]:
submission.to_csv('submission.csv', index=False)